In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, split, explode, lower, regexp_replace, collect_set, size, array_intersect, array_union, lit, when
from pyspark.sql.window import Window
import socket
import time

# --- Configuração do Spark Session ---
local_ip = socket.gethostbyname(socket.gethostname())

spark = SparkSession.builder \
    .appName("Workload Final - Similaridade Jaccard de Artistas") \
    .master("spark://spark-master:7077") \
    .getOrCreate()

print("Spark Session iniciada com sucesso!")

def executar_workload_final():
    """
    Executa um workload de estresse para encontrar os pares de artistas mais similares
    dentro dos principais gêneros, usando a Similaridade Jaccard.
    """
    # --- 1. Carregar e Preparar os Dados ---
    caminho_parquet = "/spark-data/musicas_limpas_cluster.parquet"
    df = spark.read.parquet(caminho_parquet)

    # Filtra para os 5 gêneros mais populares para o teste. Aumentar este número aumenta drasticamente a carga.
    top_genres = [row['main_genre'] for row in df.groupBy("main_genre").count().orderBy(col("count").desc()).limit(5).collect()]
    df_filtrado = df.filter(col("main_genre").isin(top_genres))

    # --- 2. Processamento e Agregação de Palavras por Artista ---
    print("Etapa 1/4: Criando vocabulário por artista...")
    df_palavras = df_filtrado.withColumn("palavra", explode(split(lower(regexp_replace(col("text"), r'[\W_]+', ' ')), ' '))) \
                             .select("main_genre", "Artist(s)", "palavra") \
                             .filter(col("palavra") != "")

    df_vocabulario_artista = df_palavras.groupBy("main_genre", "Artist(s)").agg(collect_set("palavra").alias("vocabulario"))
    
    # O cache é crucial aqui para evitar que o Spark recalcule o vocabulário a cada iteração do join.
    df_vocabulario_artista.cache()
    print(f"Vocabulário criado para {df_vocabulario_artista.count()} artistas.")

    # --- 3. Self-Join e Cálculo da Similaridade Jaccard (A PARTE MAIS PESADA) ---
    print("Etapa 2/4: Realizando cross-join e calculando similaridade (esta etapa pode demorar)...")
    df_a1 = df_vocabulario_artista.alias("a1")
    df_a2 = df_vocabulario_artista.alias("a2")

    # Realiza o join para criar pares de artistas distintos DENTRO do mesmo gênero
    df_pares_artistas = df_a1.join(
        df_a2,
        (col("a1.main_genre") == col("a2.main_genre")) & (col("a1.`Artist(s)`") < col("a2.`Artist(s)`")),
        "inner"
    )

    # Calcula a interseção e a união para cada par
    df_jaccard = df_pares_artistas.withColumn(
        "intersecao",
        size(array_intersect(col("a1.vocabulario"), col("a2.vocabulario")))
    ).withColumn(
        "uniao",
        size(array_union(col("a1.vocabulario"), col("a2.vocabulario")))
    ).withColumn(
        "similaridade_jaccard",
        # Evita divisão por zero se a união for 0
        when(col("uniao") == 0, 0.0).otherwise(col("intersecao") / col("uniao"))
    )

    # --- 4. Encontrar os Pares Mais Similares por Gênero ---
    print("Etapa 3/4: Ranqueando os resultados...")
    window_spec = Window.partitionBy(col("a1.main_genre")).orderBy(col("similaridade_jaccard").desc())
    
    df_resultado = df_jaccard.withColumn("rank", col("intersecao").desc()) \
                             .select(
                                 col("a1.main_genre").alias("genero"),
                                 col("a1.`Artist(s)`").alias("artista_1"),
                                 col("a2.`Artist(s)`").alias("artista_2"),
                                 col("similaridade_jaccard")
                             ) \
                             .orderBy(col("similaridade_jaccard").desc())

    # --- 5. Salvar Resultado ---
    print("Etapa 4/4: Salvando os resultados...")
    caminho_saida = "/spark-data/resultados_similaridade_jaccard"

    print(f"Workload concluído. Top 100 resultados")
    #df_resultado.show(20, truncate=False)


# --- Medição do Tempo de Execução ---
print("Iniciando a execução do Workload Final de Estresse...")
inicio = time.time()

executar_workload_final()

fim = time.time()
duracao = fim - inicio
throughput = 424/duracao

print(f"\n--- Análise de Desempenho ---")
print(f"Tempo total de execução: {duracao:.2f} segundos")
print(f"Throughput da execução: {throughput:.2f} MB/s")
print(f"----------------------------")

spark.stop()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/09 20:55:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark Session iniciada com sucesso!
Iniciando a execução do Workload Final de Estresse...


Etapa 1/4: Criando vocabulário por artista...


Vocabulário criado para 114163 artistas.
Etapa 2/4: Realizando cross-join e calculando similaridade (esta etapa pode demorar)...
Etapa 3/4: Ranqueando os resultados...
Etapa 4/4: Salvando os resultados...
Workload concluído. Top 100 resultados

--- Análise de Desempenho ---
Tempo total de execução: 53.17 segundos
Throughput da execução: 7.98 MB/s
----------------------------


In [ ]:
spark.stop()